In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline
def sol() :
  N, M = map(int, input().split())
  G = [[] for _ in range(N+1)]
  for i, p in enumerate(map(int, input().split())) :
    if p == -1 : continue
    G[p].append(i+1)
  
  S = [0] * (N+1) #i번 정점에 방문한 시점
  E = [0] * (N+1) #i번 정점을 빠져나온 시점
  cnt = 0

  ST = [(1, -1)]
  while ST:
    x, p = ST[-1]

    if S[x] == 0:
      cnt += 1
      S[x] = cnt

      for nx in G[x]:
        if nx == p: continue
        ST.append((nx, x))
    else:
      ST.pop()
      E[x] = cnt

  dp = [0] * (N+1)
  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 1 :
      x, w = l
      for i in range(S[x], E[x]+1) :
        dp[i] += w
    elif q == 2 :
      x = l[0]
      ans.append(dp[S[x]])

  sys.stdout.write('\n'.join(map(str, ans)))
  
sol()

### 자력솔
- 풀이는 16404와 같다. Lazy가 없어도 풀린다는걸 보고 시도해본 풀이.
- for문으로 그냥 무지성으로 더하는건 pypy로 15000ms가 걸렸다. 말그대로 턱걸이.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class SegmentTree:
  def __init__(self, L, default, f):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, i):
    self[i] = self._def

  def __getitem__(self, i):
    return self.L[i + self._size]

  def __setitem__(self, i, v):
    i += self._size
    self.L[i] = v
    i >>= 1
    while i:
      self.L[i] = self._f(self.L[2 * i], self.L[2 * i + 1])
      i >>= 1

  def __len__(self):
    return self._len

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

def sol() :
  N, M = map(int, input().split())
  G = [[] for _ in range(N+1)]
  for i, p in enumerate(map(int, input().split())) :
    if p == -1 : continue
    G[p].append(i+1)
  
  S = [0] * (N+1) #i번 정점에 방문한 시점
  E = [0] * (N+1) #i번 정점을 빠져나온 시점
  cnt = 0

  ST = [(1, -1)]
  while ST:
    x, p = ST[-1]

    if S[x] == 0:
      cnt += 1
      S[x] = cnt

      for nx in G[x]:
        if nx == p: continue
        ST.append((nx, x))
    else:
      ST.pop()
      E[x] = cnt

  seg = SegmentTree([0] * (N+10), 0, lambda x, y : x + y)
  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 1 :
      x, w = l
      seg[S[x]] += w
      seg[E[x]+1] -= w
    elif q == 2 :
      x = l[0]
      ans.append(seg.query(0, S[x]+1))

  sys.stdout.write('\n'.join(map(str, ans)))
  
sol()

- imos법을 세그먼트로 구현한게 정답이였다. 이게 가장 빠른 시간에 풀린다.